# Grid world

We define a game where a robot learns to find an optimal path in a room to a target while avoiding traps. 

The room is described by a 3x4 grid. Initially the robot is in (2,0), the target is in (0,3), the trap in (1,3) and the cell (1,1) is unreachable (wall or pillar).

```
    +---+---+---+---+
    |   |   |   | +1|
    +---+---+---+---+
    |   |XXX|   | -1|
    +---+---+---+---+
    | R |   |   |   |
    +---+---+---+---+
    
```
The actions available to the robot are go North, South, East and West (N, S, E, W).

This problem exposes 11 reachable states and 4 possible actions.

In [1]:
import numpy as np


The class `Grid` describes a generic room (size, rewards for each cell and actions possible from each cell) and keeps the current state (the position of the agent).

In [18]:
class Grid:
    def __init__(self, height, width):
        self.width = width
        self.height = height
        
    def configure(self, rewards, actions, transitions, start):
        self.rewards = rewards
        self.actions = actions
        self.transitions = transitions
        self.start = start
        self.restart()
        
    def restart(self, start=None, random=False):
        if random: 
            K = list(self.actions.keys())
            start = K[np.random.choice(len(K))]
        elif start is None: 
            start = self.start
        self.state = start
        
    def is_terminal_state(self, s):
        return s not in self.actions
    
    def is_game_over(self):
        return self.is_terminal_state(self.state)
    
    def all_states(self):
        for r in range(self.height):
            for c in range(self.width):
                yield (r,c)
    
    def all_actions(self):
        return list(set(s for a in std_grid.actions.values() for s in a))
    
    def move(self, a):
        if self.is_game_over(): return 0.0
        
        # get the transition distribution from the current state with action a
        targets = self.transitions(self.state, a)        
        if not targets: return 0 # move is not possible
        
        print (targets)
        K = list(targets.keys())
        V = list(targets.values())
        self.state = K[np.random.choice(len(K), p=V)]
        
        return self.rewards.get(self.state, 0.0)

Let's now define a standard grid factory function. It creates a new `Grid` instance and configure it as described above. 

In [3]:
def build_standard_grid():
    g = Grid(3,4)
    
    rewards = { (0,3): 1.0, 
                (1,3): -1.0
              }
    
    actions = { (0,0): ['E', 'S'],
                (0,1): ['W', 'E'],
                (0,2): ['W', 'E', 'S'],
                (1,0): ['N', 'S'],
                (1,2): ['E', 'N', 'S'],
                (2,0): ['E', 'N'],
                (2,1): ['W', 'E'],
                (2,2): ['W', 'E', 'N'],
                (2,3): ['W', 'N']
              }
    
    def transitions(s, a):
        if s not in actions:return {}
        if a not in actions[s]: return {}
        if a=='S': return {(s[0]+1, s[1]  ): 1.0}
        if a=='N': return {(s[0]-1, s[1]  ): 1.0}
        if a=='E': return {(s[0]  , s[1]+1): 1.0}
        if a=='W': return {(s[0]  , s[1]-1): 1.0}

    
    g.configure(rewards, actions, transitions, (2,0))
    return g

We will also want to inspect the differents solutions we have found. The following functions display the value function and the policy on the grid.

In [4]:
def print_value(V, g):
    print ("Value function")
    for c in range(g.width):
        print("+-------", end='')
    print("+")
    for r in range(g.height):
        for c in range(g.width):
            print (f"| {V.get((r,c),0):+0.2f} ", end='')
        print("|")
        for c in range(g.width):
            print("+-------", end='')
        print("+")
        
        
def print_policy(pi, g):
    print ("Policy")
    
    for c in range(g.width):
        print("+-------------", end='')
    print("+")
    
    for r in range(g.height):
        for c in range(g.width):
            s = (r,c)
            print (f"|    {pi(g, 'N', s):+0.2f}    ", end='')
        print("|")
        for c in range(g.width):
            s = (r,c)
            print (f"| {pi(g, 'W', s):+0.2f} {pi(g, 'E', s):+0.2f} ", end='')
        print("|")
        for c in range(g.width):
            s = (r,c)
            print (f"|    {pi(g, 'S', s):+0.2f}    ", end='')
        print("|")
        for c in range(g.width):
            print("+-------------", end='')
        print("+")



## Prediction Problem
Given a policy, we want to find the value function $V^\pi(s)$.

### Iterative Policy Evaluation

For that we will use the **Iterative Policy Evaluation** algorithm which loops repetitively to update the value function at each state using the Bellman equation until it converges.

The Bellman equation compute the value of a state from the value of the future possible states, according to a policy:

$$V^\pi(s) = \sum_{a}{\pi(a \mid s) \sum_{s'}\sum_{r}{p(s',r \mid s,a)(r + \gamma V^\pi(s'))}}$$

where:

- $V^\pi(s)$ is the value of the state $s$ according to the policy $\pi$
- $\pi(a \mid s)$ is the policy. It gives the probability of taking the action $a$ given we are in state $s$
- $p(s',r \mid s,a)$ is the probability of transitionning to state $s'$ and getting the reward $r$, while we are in state $s$ and take the action $a$. In our case, transitions from state to state are deterministic, so this value is always $1$ or $0$.
- $r$ is the reward of the transition $s \rightarrow s'$
- $\gamma$ is the discount factor

Initially, all values are set to zero. Then we iterate continuously over all states, applying the Bellman equation to compute the value at each state. The loop finishes when all the values converge toward a stable value (which mean the difference between the new computed value and the previous one is very small).


In [5]:
def iterative_policy_evaluation(grid, pi, gamma, debug=False):
    
    # initialize the value function to 0
    V = { s:0.0 for s in grid.all_states() }
    
    
    # loop until stabilization
    loop = 0
    while True:
        if debug: print (f"--- Loop #{loop}")
        loop += 1
        
        delta = 0
        
        # enumerate all states
        for s in grid.all_states():
            if debug: print (f"   State {s}")
            
            # if terminal, the value is still zero
            if grid.is_terminal_state(s):
                if debug: print (f"   -> terminal")
                continue
                
                
            # Bellman equation : sum over all available actions
            v = 0
            for a in grid.actions[s]:
                if debug: print (f"      Action: {a}")
                
                # get the probability of taking action 'a' while in state 's'
                pa = pi(grid, a, s)
                if debug: print (f"          pa={pa}")
                if pa == 0: continue
                
                # get the distribution of possible targets if we take action 'a' from state 's'
                targets = grid.transitions(s,a)
                if debug: print (f"          targets={targets}")
                if not targets: continue
                
                for next_state, prob in targets.items():
                    if debug: print (f"              next_state={next_state}, prob={prob}")
                    if debug: print (f"              V[next_state]={V[next_state]}")
                    v += pa * prob * (grid.rewards.get(next_state, 0.0) + gamma * V[next_state])

            
            delta = max(delta, abs(v - V[s]))
            if debug: print (f"   Vs={V[s]} -> {v} : delta={delta}")
                    
            V[s] = v
            
        if (delta < 1e-3):
            break
                
    return V

### Random policy
To begin, we evaluate a random policy. This policy states that we have the same probability to take any available action at every state. More formally, at each state $s$, there is a set of available action $A_s = {a_0, a_1, \ldots , a_N }$. The probability to take the action $a_i$ is $p(a_i) = 1\,/\,|A_s|$

In [6]:
def random_policy(grid, a, s):
    if s not in grid.actions: return 0.0
    if a not in grid.actions[s]: return 0.0
    return 1.0 / len(grid.actions[s])



std_grid = build_standard_grid()

V = iterative_policy_evaluation(std_grid, random_policy, 1.0)

print_policy(random_policy, std_grid)
print_value (V, std_grid)


Policy
+-------------+-------------+-------------+-------------+
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
| +0.00 +0.50 | +0.50 +0.50 | +0.33 +0.33 | +0.00 +0.00 |
|    +0.50    |    +0.00    |    +0.33    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +0.50    |    +0.00    |    +0.33    |    +0.00    |
| +0.00 +0.00 | +0.00 +0.00 | +0.00 +0.33 | +0.00 +0.00 |
|    +0.50    |    +0.00    |    +0.33    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +0.50    |    +0.00    |    +0.33    |    +0.50    |
| +0.00 +0.50 | +0.50 +0.50 | +0.33 +0.33 | +0.50 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
Value function
+-------+-------+-------+-------+
| -0.03 | +0.09 | +0.22 | +0.00 |
+-------+-------+-------+-------+
| -0.16 | +0.00 | -0.44 | +0.00 |
+-------+-------+-------+-------+
| -0.29 | -0.41 | -0.54 | -0.77 |
+-------+-------+---

### Fixed policy
The next policy we want to evaluate is a fixed policy. At each state, the action is fixed. For our example, any state on the left column or the top row will lead to the target, and any other state will lead to the trap.

```
    +---+---+---+---+
    | → | → | → | +1|
    +---+---+---+---+
    | ↑ |XXX| → | -1|
    +---+---+---+---+
    | ↑ | → | → | ↑ |
    +---+---+---+---+
    
```

In [7]:
def fixed_policy(grid, a, s):
    fixed = { (0,0):'E', (0,1):'E', (0,2):'E', 
              (1,0):'N',            (1,2):'E',  
              (2,0):'N', (2,1):'E', (2,2):'E', (2,3):'N',  
            }
    return 1.0 if s in fixed and fixed[s] == a else 0.0


V = iterative_policy_evaluation(std_grid, fixed_policy, 0.9)

print_policy(fixed_policy, std_grid)
print_value (V, std_grid)


Policy
+-------------+-------------+-------------+-------------+
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
| +0.00 +1.00 | +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +1.00    |    +0.00    |    +0.00    |    +0.00    |
| +0.00 +0.00 | +0.00 +0.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +1.00    |    +0.00    |    +0.00    |    +1.00    |
| +0.00 +0.00 | +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
Value function
+-------+-------+-------+-------+
| +0.81 | +0.90 | +1.00 | +0.00 |
+-------+-------+-------+-------+
| +0.73 | +0.00 | -1.00 | +0.00 |
+-------+-------+-------+-------+
| +0.66 | -0.81 | -0.90 | -1.00 |
+-------+-------+---

## Policy improvement : control problem

Many RL algorithms are based on estimating value functions that estimate how good it is for the agent to be in a given state (or how good it is to perform a given action in a given state). 

A policy's value function assigns to each state the expected return from that state given that the agent uses the policy. The optimal value function assigns to each state the largest expected return achievable by any policy. A policy whose value function is the optimal value function is an optimal policy.

With the previous algorithm, we have a way to find the value function of the game given a policy. In this part, we will see how we can find the optimal policy for this problem: this a a _control problem_.

The value of a state $V^\pi(s)$ (_state-value_ function) is the expected return starting from that state $s$. It depends on the agent’s policy:

$$
\begin{split}
V^\pi(s) & = E_\pi[G(t) \mid s] \\
         & = \sum_{a}{\pi(a \mid s) \sum_{s'}\sum_{r}{p(s',r \mid s,a)(r + \gamma V^\pi(s'))}}
\end{split}
$$

The value of taking the action in a state under the policy $Q^\pi(s,a)$ (_action-value_ function) is the expected return starting from that state $s$, taking that action $a$, and thereafter following the policy $\pi$:

$$Q^\pi(s,a) = \sum_{s'}\sum_{r}{p(s',r \mid s,a)(r + \gamma V^\pi(s'))}$$

The difference is subtile but this will help us optimizing the policy. Using the current policy, we simply get $V^\pi(s)$. To optimize this policy, we need to change some actions in this policy and see if taking this action improve the reward expectation ($V^{\pi'}(s) > V^\pi(s)$). As for each state we have a finite set of actions (U, D, L, R), we just go through each one until we get a better value. More formally, that is find $a \in A$ s.t. $Q^\pi(s,a) > Q^\pi(s, \pi(s))$.

We are finding a new policy $\pi'$ that gives us a bigger value than we had before: $ V^\pi(s) \leq V^{\pi'}(s) $

$$
\begin{split}
\pi'(s) & = \operatorname{argmax}_a Q^\pi(s,a) \\
        & = \operatorname{argmax}_a \sum_{s'}\sum_{r}{p(s',r \mid s,a)(r + \gamma V^\pi(s'))}
\end{split}
$$


### Policy iteration

This algorithm alternates between policy evaluation (compute the value fonction under the current policy) and policy improvement (find the optimal actions to take) and continues until the policy doesn't change anymore.

In [8]:
def build_fixed_policy(state_action_map):
    return lambda grid, a, s: 1.0 if s in state_action_map and state_action_map[s] == a else 0.0

def policy_iteration(grid, gamma, debug=False):
    # randomly initialize the actions at each state
    V = { s:np.random.random() for s in grid.all_states() }
    A = { s:np.random.choice([a for a in grid.all_actions()]) for s in grid.all_states() }
    
    # create a policy function
    pi = build_fixed_policy(A)

    loop = 0
    while True:
        if debug: print (f"Loop #{loop}: A={A}")
        loop+=1
        
        # policy evaluation
        V = iterative_policy_evaluation(grid, pi, 0.9)
        if debug: print (f"         V={V}")
        
        # policy improvement
        policy_changed = False
        for s in grid.all_states():
            if debug: print (f"    State {s}")
            
            # init max with current values
            maxV = float('-inf')
            maxA = ''
                        
            for a in grid.all_actions():
                
                # get p(s′,r|s,a), the distribution of possible targets if we take action 'a' from state 's'
                targets = grid.transitions(s,a)
                if not targets: continue
                
                v = 0
                for next_state, prob in targets.items():
                    v += prob * (grid.rewards.get(next_state, 0.0) + gamma * V[next_state])
                
                if debug: print (f"      {a} -> {v}")
                if v >= maxV:
                    maxV = v
                    maxA = a

            if maxA != A[s]:
                A[s] = maxA
                policy_changed = True
            
        if not policy_changed:
            break
            
    # last value evaluation
    V = iterative_policy_evaluation(grid, pi, 0.9)

    return A, V



We build a new grid which penalizes each move with a negative reward (for example to simulate the robot power usage). All non terminal states will have the same cost, except the (0,0) that will all twice the cost of all others to simulate an area where it is more difficult to move (sand, water, ...). The optimal policy should eventually avoid this state.

In [9]:
def build_negative_grid(step_cost=-0.1):
    g = build_standard_grid()
    g.rewards.update({ s:step_cost for s in g.actions.keys() })
    g.rewards.update({ (0,0):2*step_cost })
    return g
    
neg_grid = build_negative_grid(-0.1)

A,V = policy_iteration(neg_grid, 0.9)

print_value (neg_grid.rewards, neg_grid)
print_policy(build_fixed_policy(A), neg_grid)
print_value (V, neg_grid)

Value function
+-------+-------+-------+-------+
| -0.20 | -0.10 | -0.10 | +1.00 |
+-------+-------+-------+-------+
| -0.10 | +0.00 | -0.10 | -1.00 |
+-------+-------+-------+-------+
| -0.10 | -0.10 | -0.10 | -0.10 |
+-------+-------+-------+-------+
Policy
+-------------+-------------+-------------+-------------+
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
| +0.00 +1.00 | +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +1.00    |    +0.00    |    +1.00    |    +0.00    |
| +0.00 +0.00 | +0.00 +0.00 | +0.00 +0.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +0.00    |    +0.00    |    +1.00    |    +0.00    |
| +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 | +1.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-

### Windy grid world
So far, the transitions $p(s',r \mid s,a)$ has been deterministic. Let's add a bit of randomness in the grid world and suppose the agent actions can be disturbed by violent gusts of wind. The probability that the expected action occurs is 0.5, but another action can also occur with a probability of 0.5/3.

In [19]:
def build_windy_grid(step_cost=-0.1):
    g = build_negative_grid(step_cost)
    
    def transitions(s, a):
        
        if s not in g.actions: return {}
        if a not in g.actions[s]: return {}
        
        n = len(g.actions[s]) - 1
        
        go_s = ( min(g.height-1,s[0]+1), s[1]                )
        go_n = ( max(0, s[0]-1)        , s[1]                )
        go_e = ( s[0]                  , min(g.width-1,s[1]+1) )
        go_w = ( s[0]                  , max(0, s[1]-1)      )
        
        if a=='S': return {go_s: 0.5 , go_n:0.5/3, go_e:0.5/3, go_w:0.5/3 }
        if a=='N': return {go_s:0.5/3, go_n: 0.5 , go_e:0.5/3, go_w:0.5/3 }
        if a=='E': return {go_s:0.5/3, go_n:0.5/3, go_e: 0.5 , go_w:0.5/3 }
        if a=='W': return {go_s:0.5/3, go_n:0.5/3, go_e:0.5/3, go_w: 0.5  }

    g.transitions = transitions
    return g

wind_grid = build_windy_grid()

A,V = policy_iteration(wind_grid, 0.9, debug=False)

print_value (wind_grid.rewards, wind_grid)
print_policy(build_fixed_policy(A), wind_grid)
print_value (V, wind_grid)

Value function
+-------+-------+-------+-------+
| -0.20 | -0.10 | -0.10 | +1.00 |
+-------+-------+-------+-------+
| -0.10 | +0.00 | -0.10 | -1.00 |
+-------+-------+-------+-------+
| -0.10 | -0.10 | -0.10 | -0.10 |
+-------+-------+-------+-------+
Policy
+-------------+-------------+-------------+-------------+
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
| +0.00 +1.00 | +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +1.00    |    +0.00    |    +1.00    |    +0.00    |
| +0.00 +0.00 | +0.00 +0.00 | +0.00 +0.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +1.00    |    +0.00    |    +1.00    |    +0.00    |
| +0.00 +0.00 | +0.00 +1.00 | +0.00 +0.00 | +1.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-

We can see that, although the state (0,0) has a greater cost than the other states, the optimal policy pass through it. This is probably to avoid passing next to the trap where the wind has chances to push the agent.

### Value iteration

The value iteration algorithm is a variant of the policy iteration that combines policy evaluation and policy improvement into one step:

$$V^{\pi'}(s) = \operatorname{max}_a \sum_{s'}\sum_{r}{p(s',r \mid s,a)(r + \gamma V^\pi(s'))}$$



In [11]:
def value_iteration(grid, gamma, debug=False):
    
    # initialize the value function to 0
    V = { s:0.0 for s in grid.all_states() }
    
    
    # loop until stabilization
    loop = 0
    while True:
        if debug: print (f"--- Loop #{loop}")
        loop += 1
        
        delta = 0
        
        # enumerate all states
        for s in grid.all_states():
            if debug: print (f"   State {s}")
            
            # if terminal, the value is still zero
            if grid.is_terminal_state(s):
                if debug: print (f"   -> terminal")
                continue
                
                
            maxV = float('-inf')
            for a in grid.all_actions():
                
                # get p(s′,r|s,a), the distribution of possible targets if we take action 'a' from state 's'
                targets = grid.transitions(s,a)
                if not targets: continue
                
                v = 0
                for next_state, prob in targets.items():
                    v += prob * (grid.rewards.get(next_state, 0.0) + gamma * V[next_state])
                
                if debug: print (f"      {a} -> {v}")
                if v >= maxV:
                    maxV = v
            
            delta = max(delta, abs(maxV - V[s]))
            if debug: print (f"   Vs={V[s]} -> {maxV} : delta={delta}")
                    
            V[s] = maxV
            
        if (delta < 1e-3):
            break
    
    for s in grid.all_states():
        if debug: print (f"    State {s}")

        maxV = float('-inf')
        maxA = ''

        for a in grid.all_actions():

            # get p(s′,r|s,a), the distribution of possible targets if we take action 'a' from state 's'
            targets = grid.transitions(s,a)
            if not targets: continue

            v = 0
            for next_state, prob in targets.items():
                v += prob * (grid.rewards.get(next_state, 0.0) + gamma * V[next_state])

            if debug: print (f"      {a} -> {v}")
            if v >= maxV:
                maxV = v
                maxA = a

        A[s] = maxA

    return A, V


In [12]:
A,V = value_iteration(neg_grid, 0.9, debug=False)

print_value (neg_grid.rewards, neg_grid)
print_policy(build_fixed_policy(A), neg_grid)
print_value (V, neg_grid)

Value function
+-------+-------+-------+-------+
| -0.20 | -0.10 | -0.10 | +1.00 |
+-------+-------+-------+-------+
| -0.10 | +0.00 | -0.10 | -1.00 |
+-------+-------+-------+-------+
| -0.10 | -0.10 | -0.10 | -0.10 |
+-------+-------+-------+-------+
Policy
+-------------+-------------+-------------+-------------+
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
| +0.00 +1.00 | +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +1.00    |    +0.00    |    +1.00    |    +0.00    |
| +0.00 +0.00 | +0.00 +0.00 | +0.00 +0.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +0.00    |    +0.00    |    +1.00    |    +0.00    |
| +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 | +1.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-

The value iteration algorithm should be more performant than the policy iteration algorithm. And indeed, we observe a factor 10.

In [13]:
%timeit policy_iteration(neg_grid, 0.9, debug=False)
%timeit value_iteration(neg_grid, 0.9, debug=False)

2.95 ms ± 429 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
389 µs ± 39.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Monte-Carlo method

Dynamic Programming supposes that we have access to the full model of the environment; oncluding the transition rules $p(s',r|s,a)$. But, most of the time, this is not the case et we introduce here the Monte-Carlo algorithm which is designed to learn purely from experience.

MC methods are a broad class of algorithms that rely on repeated random sampling to obtain numerical results.

in MC, instead of computing the _true_ expected value of $G$ at each state $s$, we calculate a _sample mean_ from a number of episodes $N$:

$$\bar V^\pi(s) = \frac{1}{N}\sum_{i=1}^{N}G_{i,s}$$

$G$ is generated by playing $N$ episodes, log the states and reward sequences, and use the recursive definition of $G(t)$:

$$G(t) = r(t+1) + \gamma G(t+1)$$

Then, once we have $\{s,G\}$ pairs, we average them for each state.

In the case where we see the same state more than once in an episode, we have 2 choices: either we use only the first and discard the others (this is called "first-visit MC"), or we use all values as different samples ("every-visit MC"). Surprisingly, both methods lead to the same results, so we will use only the first-visit method.

In [14]:
def first_visit_mc_predition(grid, pi, N, gamma):

    states_and_returns = {} # key:state - value:array of returns
    V = {}
    
    for i in range(N):
        snr = play_episode(grid, pi, gamma)
        seen_states = set()
        for (s,g) in snr:
            if s not in seen_states:
                seen_states.add(s)
                if s in states_and_returns:
                    states_and_returns[s].append(g)
                else:
                    states_and_returns[s] = [g]
        
        for s, returns in states_and_returns.items():
            V[s] = np.mean(returns)
            
    return V

In [21]:
# select an action to do from state s according to the policy pi
def select_action(grid, pi, s):
    A = grid.actions[s]
    p = [pi(grid, a, s) for a in A]
    return np.random.choice(A, p=p)
    

def play_episode(grid, pi, gamma):
    grid.restart(random=True)
    
    # play until gme is over
    s = grid.state 
    states_and_rewards = [(s,0)]
    while not grid.is_game_over():    
        a = select_action(grid, pi, s)
        print(s,"->", a)
        r = grid.move(a)
        s = grid.state
        states_and_rewards.append((s,r))
        
    # iteratively compute returns from rewards
    G = 0
    states_and_returns = []
    for s,r in reversed(states_and_rewards):
        states_and_returns.append((s,G))
        G = r + gamma * G

    states_and_returns.reverse()

    return states_and_returns

We first test this with the fixed policy, and we should retrieve the same value function as above.

In [16]:
V = first_visit_mc_predition(std_grid, fixed_policy, 100, 0.9)

print_policy(fixed_policy, std_grid)
print_value (V, std_grid)


Policy
+-------------+-------------+-------------+-------------+
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
| +0.00 +1.00 | +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +1.00    |    +0.00    |    +0.00    |    +0.00    |
| +0.00 +0.00 | +0.00 +0.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
|    +1.00    |    +0.00    |    +0.00    |    +1.00    |
| +0.00 +0.00 | +0.00 +1.00 | +0.00 +1.00 | +0.00 +0.00 |
|    +0.00    |    +0.00    |    +0.00    |    +0.00    |
+-------------+-------------+-------------+-------------+
Value function
+-------+-------+-------+-------+
| +0.81 | +0.90 | +1.00 | +0.00 |
+-------+-------+-------+-------+
| +0.73 | +0.00 | -1.00 | +0.00 |
+-------+-------+-------+-------+
| +0.66 | -0.81 | -0.90 | -1.00 |
+-------+-------+---

This standard grid is fully deterministic as the target state is completely determined by the current state and the action taken. In this case, MC is not really needed and does not give better results than DP. We will now use the windy grid that adds a bit of randomness in the target state computation.

In [23]:
V = first_visit_mc_predition(wind_grid, fixed_policy, 100, 0.9)

print_policy(fixed_policy, wind_grid)
print_value (V, wind_grid)


(2, 1) -> E
{(2, 1): 0.16666666666666666, (1, 1): 0.16666666666666666, (2, 2): 0.5, (2, 0): 0.16666666666666666}
(2, 2) -> E
{(2, 2): 0.16666666666666666, (1, 2): 0.16666666666666666, (2, 3): 0.5, (2, 1): 0.16666666666666666}
(2, 1) -> E
{(2, 1): 0.16666666666666666, (1, 1): 0.16666666666666666, (2, 2): 0.5, (2, 0): 0.16666666666666666}
(2, 0) -> N
{(2, 0): 0.16666666666666666, (1, 0): 0.5, (2, 1): 0.16666666666666666}


ValueError: probabilities do not sum to 1